# **Q2**


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
from google.colab import files
uploaded=files.upload()

Saving data_q2_q3.xlsx to data_q2_q3.xlsx


In [ ]:
def norm1(x):
  return (x-np.mean(x,axis=0))/np.std(x,axis=0)
def norm2(x):
  return x/np.max(x,axis=0)
def hyp(x,w):
  return np.dot(x,w)
def mse(y,pred,l):
  return (1/l)*np.sum(np.square(y-pred))
def mae(y,pred,l):
  return (1/l)*np.sum(np.absolute(y-pred))
def cc(y,pred):
  return (np.sum(np.abs(np.multiply(y,pred))))/(np.sqrt(np.sum(np.square(y))*np.sum(np.square(pred))))

In [ ]:
data=pd.read_excel("/content/data_q2_q3.xlsx")
data.insert(0,"x0",1)

In [ ]:
f,tr,va,te = 5,80,12,23
data_tr=data.loc[:79,:].copy()
data_tr=np.array(data_tr)
norm = np.amax(np.abs(data_tr),axis=0)
data_tr=data_tr/norm
data_va=data.loc[80:91,:].copy()
data_va=np.array(data_va)
data_te=data.loc[92:,:].copy()
data_te=np.array(data_te)
data_va,data_te = data_va/norm,data_te/norm
x_train,y_train,x_val,y_val,x_test,y_test=data_tr[:,:f],data_tr[:,f].reshape(tr,1),data_va[:,:f],data_va[:,f].reshape(va,1),data_te[:,:f],data_te[:,f].reshape(te,1)

## BGD

In [ ]:
# for grid search
T_grid,A_grid,L_grid=np.linspace(300,950,10),np.linspace(0.1,1,10),np.linspace(0.01,0.1,10)
params=[0,0,0] # T_ideal,Alpha,Lambda

In [ ]:
curr_err=100 # current error

for T in T_grid:
  for alpha in A_grid:
    for lamb in L_grid:
      w=np.random.rand(f).reshape(f,1) # weight vector initialised
      for t in range(int(T)):
        j1 = hyp(x_train,w)-y_train # (hyp(xi)-yi)
        for j in range(f):
          jsum = np.sum(np.multiply(j1,x_train[:,j].reshape(tr,1)))
          w[j] = (1-(lamb*alpha))*w[j] - (1/tr)*alpha*jsum
      mse_val = mse(hyp(x_val,w),y_val,va)
      if mse_val < curr_err:
        curr_err = mse_val
        params = [int(T),alpha,lamb]

In [ ]:
T_ideal,Alpha,Lambda=params[0],params[1],params[2]
params

[444, 0.1, 0.01]

In [ ]:
w=np.random.rand(f).reshape(f,1) # weight vector initialised

for t in range(T_ideal):
    j1 = hyp(x_train,w)-y_train # (hyp(xi)-yi)
    for j in range(f):
      j2 = np.multiply(j1,x_train[:,j].reshape(tr,1))
      jsum = np.sum(j2)
      w[j] = (1-(Lambda*Alpha))*w[j] - (1/tr)*Alpha*jsum

print("yp = "+str(w[0])+" + "+str(w[1])+"(x1) + "+str(w[2])+"(x2) + "+str(w[3])+"(x3) + "+str(w[4])+"(x4)")

yp = [0.03283477] + [0.58517048](x1) + [0.01172518](x2) + [0.04059653](x3) + [0.43055146](x4)


In [ ]:
# testing
predicted_output = hyp(x_test,w)
predicted_output*norm[-1]

In [ ]:
# Error calculation
print("Mean Square Error = "+str(mse(y_test,predicted_output,te)))
print("Mean Absolute Error = "+str(mae(y_test,predicted_output,te)))
print("Correlation Coefficient = "+str(cc(y_test,predicted_output)))

Mean Square Error = 0.04420057204439927
Mean Absolute Error = 0.18435975626443937
Correlation Coefficient = 0.9860478195904131


## SGD


In [ ]:
# for grid search
T_grid,A_grid,L_grid=np.linspace(200,380,10),np.linspace(0.1,1,10),np.linspace(0.01,0.1,10)
params=[0,0,0] # T_ideal,Alpha,Lambda

In [ ]:
curr_err=100 # current error

for T in T_grid:
  for alpha in A_grid:
    for lamb in L_grid:
      w=np.random.rand(f).reshape(f,1) # weight vector initialised
      for t in range(int(T)):
        index=np.random.randint(tr)
        j1 = (hyp(x_train,w)[index]-y_train[index]).reshape(1,1) # (hyp(xi)-yi)
        for j in range(f):
          j2 = np.multiply(j1.reshape(1,1),x_train[index,j])
          w[j] = (1-(lamb*alpha))*w[j] - alpha*j2
      mse_val = mse(hyp(x_val,w),y_val,va)
      if mse_val < curr_err:
        curr_err = mse_val
        params = [int(T),alpha,lamb]

In [ ]:
T_ideal,Alpha,Lambda=params[0],params[1],params[2]
params

[220, 0.2, 0.01]

In [ ]:
w=np.random.rand(f).reshape(f,1)

for t in range(T_ideal):
  index=np.random.randint(tr)
  j1 = hyp(x_train,w)[index]-y_train[index] # (hyp(xi)-yi)
  for j in range(f):
    j2 = np.multiply(j1.reshape(1,1),x_train[index,j])
    w[j] = (1-(Lambda*Alpha))*w[j] - Alpha*j2

print("yp = "+str(w[0])+" + "+str(w[1])+"(x1) + "+str(w[2])+"(x2) + "+str(w[3])+"(x3) + "+str(w[4])+"(x4)")

yp = [-0.00327966] + [0.56591188](x1) + [0.08099463](x2) + [0.01553209](x3) + [0.39441835](x4)


In [ ]:
# testing
predicted_output = hyp(x_test,w)
predicted_output

In [ ]:
# Error calculation
print("Mean Square Error = "+str(mse(y_test,predicted_output,te)))
print("Mean Absolute Error = "+str(mae(y_test,predicted_output,te)))
print("Correlation Coefficient = "+str(cc(y_test,predicted_output)))

Mean Square Error = 0.06357775674547785
Mean Absolute Error = 0.21727008279031879
Correlation Coefficient = 0.9823239014569547


## MGBD

In [ ]:
# for grid search
T_grid,A_grid,L_grid=np.linspace(300,560,10),np.linspace(0.1,1,10),np.linspace(0.01,0.1,10)
params=[0,0,0] # T_ideal,Alpha,Lambda
nb=20

In [ ]:
curr_err=100 # current error

for T in T_grid:
  for alpha in A_grid:
    for lamb in L_grid:
      w=np.random.rand(f).reshape(f,1) # weight vector initialised
      for t in range(int(T)):
        np.random.shuffle(data_tr)
        hypo = np.dot(data_tr[:nb,:f].reshape(nb,f),w) # hypothesis array
        j1 = np.subtract(hypo,data_tr[:nb,f].reshape(nb,1)) # (hyp(xi)-yi)
        for j in range(f):
          jsum = np.sum(np.multiply(j1,data_tr[:nb,j].reshape(nb,1)))
          w[j] = (1-(lamb*alpha))*w[j] - (1/tr)*alpha*jsum
      mse_val = mse(hyp(x_val,w),y_val,va)
      if mse_val < curr_err:
        curr_err = mse_val
        params = [int(T),alpha,lamb]

In [ ]:
T_ideal,Alpha,Lambda=params[0],params[1],params[2]
params

[300, 0.1, 0.01]

In [ ]:
w=np.random.rand(f).reshape(f,1) # weight vector initialised

for t in range(int(T_ideal)):
  np.random.shuffle(data_tr)
  hypo = np.dot(data_tr[:nb,:f].reshape(nb,f),w) # hypothesis array
  j1 = np.subtract(hypo,data_tr[:nb,f].reshape(nb,1)) # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,data_tr[:nb,j].reshape(nb,1)))
    w[j] = (1-(Lambda*Alpha))*w[j] - (1/tr)*Alpha*jsum

print("yp = "+str(w[0])+" + "+str(w[1])+"(x1) + "+str(w[2])+"(x2) + "+str(w[3])+"(x3) + "+str(w[4])+"(x4)")

yp = [0.27402186] + [0.27497585](x1) + [0.10979549](x2) + [-0.0148248](x3) + [0.35390269](x4)


In [ ]:
# testing
predicted_output = hyp(x_test,w)
predicted_output

In [ ]:
# Error calculation
print("Mean Square Error = "+str(mse(y_test,predicted_output,te)))
print("Mean Absolute Error = "+str(mae(y_test,predicted_output,te)))
print("Correlation Coefficient = "+str(cc(y_test,predicted_output)))

Mean Square Error = 0.037234168016580624
Mean Absolute Error = 0.1649562480592072
Correlation Coefficient = 0.9928109800356947
